# Jacobi to Cartesian Coordinates

In [ ]:
# Library imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Aliases
keras = tf.keras

In [22]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory
from jacobi import make_data_jacobi, make_dataset_cart_to_jac, make_dataset_jac_to_cart
from jacobi import CartesianToJacobi, JacobiToCartesian
from jacobi import make_model_cart_to_jac, make_model_jac_to_cart

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Plot style 
plot_style()

In [68]:
N = 100
num_body = 2
data = make_data_jacobi(N=N, num_body=num_body)
m = data['m']
q = data['q']
r = data['r']

In [69]:
m[0:5]

array([[1.0000000e+00, 7.9694558e-03],
       [1.0000000e+00, 1.5751318e-03],
       [1.0000000e+00, 2.0511114e-04],
       [1.0000000e+00, 5.3994847e-03],
       [1.0000000e+00, 2.6070243e-03]], dtype=float32)

In [70]:
M = np.cumsum(m, axis=-1)
M[0:5]

array([[1.       , 1.0079695],
       [1.       , 1.0015751],
       [1.       , 1.0002052],
       [1.       , 1.0053995],
       [1.       , 1.002607 ]], dtype=float32)

In [71]:
A_shape = (N, num_body, num_body)

# Assemble num_body x num_body square matrix converting from q to r
A = np.zeros(A_shape)
for n in range(N):
    A[n, 0, :] = m[n] / M[n, num_body-1]
    for i in range(1, num_body):
        for j in range(i):
            A[n, i, j] = -m[n, j] / M[n, i-1]
        A[n, i, i] = 1.0

In [72]:
A[0]

array([[ 0.9920935 ,  0.00790645],
       [-1.        ,  1.        ]])

In [73]:
# Create a tensorflow Dataset instance in both directions
N = 1024
num_body = 2
batch_size = 64

ds_c2j = make_dataset_cart_to_jac(N=N, num_body=num_body, batch_size=batch_size)
ds_j2c = make_dataset_jac_to_cart(N=N, num_body=num_body, batch_size=batch_size)

In [74]:
# Example batch - cartesian to jacobi
cart, jac = list(ds_c2j.take(1))[0]

In [75]:
# Unpack tensors
m = cart['m']
q = cart['q']
r = jac['r']

# Review shapes
print(f'Example batch sizes:')
print(f'm  = {m.shape}')
print(f'q  = {q.shape}')
print(f'r  = {r.shape}')

Example batch sizes:
m  = (64, 2)
q  = (64, 2, 3)
r  = (64, 2, 3)


In [76]:
m[0]

<tf.Tensor: id=780, shape=(2,), dtype=float32, numpy=array([1.        , 0.00208914], dtype=float32)>

In [149]:
class CartesianToJacobi(keras.layers.Layer):
    def call(self, inputs):
        """Compute Cartesian coordinate q from masses and Jacobi coordinates m, r"""
        # Unpack inputs
        # m: masses; shape (num_body)
        # q: Cartesian coordinate; shape (num_body, 3)
        m, q = inputs

        # Array shapes
        batch_size, num_body, space_dims = q.shape
        batch_size=64
        A_shape = (batch_size, num_body, num_body)
        
        # Cumulative sum of mass
        M = tf.math.cumsum(m, axis=-1)
        M_tot = keras.layers.Reshape(target_shape=(1,))(M[:, num_body-1])
        
        # Assemble num_body x num_body square matrix converting from q to r
        # Do the assembly as a numpy matrix
        A_ = np.zeros(A_shape, dtype=np.float32)
        A_[:, 0, :] = m / M_tot
        for i in range(1, num_body):
            # for j in range(i-1):
            #     A_[:, i, j] = -m[:, j] / M[:, i-1]
            A_[:, i, 0:i] = -m[:, 0:i-1] / M[:, i-1:i]
            A_[:, i, i] = 1.0

        # Now convert A to a tensor
        A = tf.Variable(A_)
        # Do the matrix multiplication in Tensorflow
        r = tf.linalg.matmul(A, q)
        
        return r

    def get_config(self):
        return dict()

In [150]:
r_ = CartesianToJacobi()([m, q])
r_.shape

TensorShape([64, 2, 3])

In [131]:
q[0]

<tf.Tensor: id=7392, shape=(2, 3), dtype=float32, numpy=
array([[-21.56563   , -11.083712  ,   0.48688745],
       [ -9.805599  ,  27.58541   ,  -1.2347783 ]], dtype=float32)>

In [133]:
m[0] / M_tot[0]

<tf.Tensor: id=7407, shape=(2,), dtype=float32, numpy=array([0.9979152 , 0.00208479], dtype=float32)>

In [130]:
r[0]

<tf.Tensor: id=7387, shape=(2, 3), dtype=float32, numpy=
array([[-21.541113  , -11.003095  ,   0.48329812],
       [ 11.760031  ,  38.66912   ,  -1.7216657 ]], dtype=float32)>

In [116]:
def make_model_cart_to_jac(num_body):
    """Model that transforms from cartesian to Jacobi coordinates"""
    # The shape shared by all the inputs
    space_dims = 3
    shape_m = (num_body,)
    shape_q = (num_body, space_dims,)

    # Create input layers    
    m = keras.Input(shape=shape_m, name='m')
    q = keras.Input(shape=shape_q, name='q')
    
    # Wrap these up into one tuple of inputs
    inputs = (m, q)

    # Calculations are in one layer that does all the work...
    r = CartesianToJacobi(name='r')(inputs)
    
    # Wrap up the outputs
    outputs = r

    # Create a model from inputs to outputs
    model = keras.Model(inputs=inputs, outputs=outputs, name='cartesian_to_jacobi')
    return model

In [117]:
# Create a model mapping cartesian to jacobi coordinates
model_c2j = make_model_cart_to_jac(num_body=num_body)

ValueError: in converted code:

    <ipython-input-114-b41714afab11>:21 call  *
        A_[:, 0, :] = m / M_tot

    ValueError: setting an array element with a sequence.


In [123]:
# Array shapes
batch_size, num_body, space_dims = q.shape
batch_size=64
A_shape = (batch_size, num_body, num_body)

# Cumulative sum of mass
M = tf.math.cumsum(m, axis=-1)
M_tot = keras.layers.Reshape(target_shape=(1,))(M[:, num_body-1])

# Assemble num_body x num_body square matrix converting from q to r
# Do the assembly as a numpy matrix
A_ = np.zeros(A_shape, dtype=np.float32)
A_[:, 0, :] = m / M_tot
for i in range(1, num_body):
    for j in range(i):
        A_[:, i, j] = -m[:, j] / M[:, i-1]
    A_[:, i, i] = 1.0

# Now convert A to a tensor
A = tf.Variable(A_)
r = tf.linalg.matmul(A, q)


In [126]:
A_[:,0,:].shape

(64, 2)

In [127]:
m / M_tot

<tf.Tensor: id=7358, shape=(64, 2), dtype=float32, numpy=
array([[9.97915208e-01, 2.08478747e-03],
       [9.99026299e-01, 9.73682909e-04],
       [9.99894738e-01, 1.05200292e-04],
       [9.93102789e-01, 6.89727999e-03],
       [9.93729353e-01, 6.27067499e-03],
       [9.99486208e-01, 5.13762992e-04],
       [9.99331653e-01, 6.68271969e-04],
       [9.99089122e-01, 9.10923642e-04],
       [9.99863148e-01, 1.36862669e-04],
       [9.96676683e-01, 3.32334545e-03],
       [9.98878419e-01, 1.12160377e-03],
       [9.99588430e-01, 4.11529589e-04],
       [9.99851823e-01, 1.48125764e-04],
       [9.99022961e-01, 9.77078103e-04],
       [9.95531261e-01, 4.46879165e-03],
       [9.98799086e-01, 1.20088656e-03],
       [9.93996024e-01, 6.00398006e-03],
       [9.93545055e-01, 6.45500142e-03],
       [9.95970905e-01, 4.02910495e-03],
       [9.97399807e-01, 2.60024541e-03],
       [9.96342480e-01, 3.65745998e-03],
       [9.98712182e-01, 1.28778070e-03],
       [9.98822033e-01, 1.17799523e-03],